In [ ]:
#!/bin/python
import glob
import pandas as pd
import numpy as np
import os
import torch
# import cPickle
import pickle
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.cluster.k_means_ import KMeans
import sys
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
import re
import time
import warnings
from pandas.core.common import SettingWithCopyWarning
from lightgbm import LGBMClassifier
import sys
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
sys.argv = ['scripts/create_feat_cnn.py', './cnn', 1280]
cnn_file_list = sys.argv[1]
n_features = sys.argv[2]

In [ ]:
path = os.path.join(cnn_file_list, '*.pkl')

filelist = []

for file in glob.glob(path):
    filelist.append(file)      

In [ ]:
# foo[0]

In [ ]:
# i=800
# foo = pickle.load(open(filelist[i],"rb"))
# list_foo = []
# foo_concat = []
# for idx in range(len(foo)):
#     list_foo.append(foo[idx].detach().numpy())
# foo_concat = np.concatenate(list_foo, axis = 0)    
# foo_concat
# # foo_concat.mean(axis = 1)    

In [ ]:
# foo_concat.shape

In [ ]:
# foo_concat.mean(axis = 0).reshape(1, 1280)

In [ ]:
# i=800
# fooo = []
# for i in range(len(filelist)):
#     foo = pickle.load(open(filelist[i],"rb"))
#     fooo.append(foo.shape[0])
#     if i % 100 == 0:
#         print(i)
#     else: pass
    

In [ ]:
# i=2129
# list_foo = []
# foo = pickle.load(open(filelist[i],"rb"))
# for idx in range(len(foo)):
#     list_foo.append(foo[idx].detach().numpy())
#     foo_concat = np.concatenate(list_foo, axis = 0)

In [ ]:
# sample_dict = dict()
knn_data_list = []
start_time = time.time()
for i in range(len(filelist)):
# for i in range(2500, 20000):
    with open(filelist[i], 'rb') as f:
        try:
            data = pickle.load(f)
            if len(data) != 0:
                list_foo = []
                foo = pickle.load(open(filelist[i],"rb"))
                for idx in range(len(foo)):
#                 for idx in srs_idx:
                    list_foo.append(foo[idx].detach().numpy())
                foo_concat = np.concatenate(list_foo, axis = 0)
                knn_data_list.append(foo_concat.mean(axis = 0).reshape(1, n_features))
            else:
                foo_concat = np.zeros((1,1280))
                knn_data_list.append(foo_concat)
        except EOFError:
            pass
    if i % 500 == 0:
        print('========== {}th step is completed! =========='.format(i))
    else:
        pass
print('========== All complete! It takes {} secs. =========='.format(time.time() - start_time))    

In [ ]:
knn_mart = np.concatenate(knn_data_list, axis = 0)

In [ ]:
data_mart = pd.DataFrame(knn_mart)
data_mart.to_csv('./total_cnn_features_avgpool.csv', index=False)

In [ ]:
total_features = pd.read_csv('./total_cnn_features_avgpool.csv')

In [ ]:
total_features.head(3)

In [ ]:
video_name_ind = []
for i in range(len(filelist)):
    match_front = re.search('cnn/', filelist[i])
    match_end = re.search('.pkl', filelist[i])
    video_name_ind.append(filelist[i][match_front.end():match_end.start()])
    video_name = pd.DataFrame({'video': video_name_ind})  

In [ ]:
# k = n_clusters
# k = total_features.shape[1]
k = n_features
column_name = ['video']
for i in range(k):
    column_name.append('feature_{}'.format(i))

In [ ]:
total_data = pd.concat([video_name, total_features], axis = 1)
total_data.columns = column_name
train_ind = pd.read_csv('./list/train', sep = ' ', header = None)
valid_ind = pd.read_csv('./list/val', sep = ' ', header = None)
test_ind = pd.read_csv('./list/test.video', sep = ' ', header = None)

In [ ]:
train_ind['Data'] = 'TRAIN'
valid_ind['Data'] = 'VALID'
test_ind[1] = 'UNK'
test_ind['Data'] = 'TEST'

In [ ]:
train_ind.columns = ['video','target','Data']
valid_ind.columns = ['video','target','Data']
test_ind.columns = ['video','target','Data']
data_lable = pd.concat([train_ind, valid_ind, test_ind], axis = 0).reset_index().drop('index', axis = 1)

In [ ]:
# data_lable['target_p001'] = 
data_lable['target_p001'] = data_lable['target']
data_lable['target_p002'] = data_lable['target']
data_lable['target_p003'] = data_lable['target']
data_lable['target_p001_10'] = 1
data_lable['target_p002_10'] = 1
data_lable['target_p003_10'] = 1

data_lable['target_p001'][data_lable['target'] != 'P001'] = 'Other'
data_lable['target_p002'][data_lable['target'] != 'P002'] = 'Other'
data_lable['target_p003'][data_lable['target'] != 'P003'] = 'Other'
data_lable['target_p001_10'][data_lable['target'] != 'P001'] = 0
data_lable['target_p002_10'][data_lable['target'] != 'P002'] = 0
data_lable['target_p003_10'][data_lable['target'] != 'P003'] = 0

In [ ]:
total_mart = total_data.merge(data_lable, how = 'right', on = 'video')
total_mart = total_mart.fillna(0)

train_mart = total_mart[total_mart['Data'] == 'TRAIN']
valid_mart = total_mart[total_mart['Data'] == 'VALID']
test_mart  = total_mart[total_mart['Data'] == 'TEST']

total_mart.to_csv('./datamart_cnn_total_avgpool.csv', index=False)
train_mart.to_csv('./datamart_cnn_train_avgpool.csv', index=False)
valid_mart.to_csv('./datamart_cnn_valid_avgpool.csv', index=False)
test_mart.to_csv('./datamart_cnn_test_avgpool.csv', index=False)

In [ ]:
def modeling_ap_SVM(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = SVC(kernel=chi2_kernel, probability=True)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of SVM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_AdaB(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = AdaBoostClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of AdaBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_Boost(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = GradientBoostingClassifier(n_estimators=200, random_state=0)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of Boosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_xgb(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of XgBoosting Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

def modeling_ap_lgbm(k, train_data, valid_data, target = 'target_p001_10'):
    start_time = time.time()
    k = k
    train_mart = train_data
    valid_mart = valid_data
    target = target
    
    X_train = train_mart.iloc[:,1:k+1]
    y_train = train_mart[target]
    X_valid = valid_mart.iloc[:,1:k+1]
    y_valid = valid_mart[target]
    
    model = LGBMClassifier(random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    y_preds = model.predict(X_valid)
    y_probs = model.predict_proba(X_valid)
    results = average_precision_score(y_true=y_valid.values, y_score=y_probs[:,1])
    print("===== The time consuming of LightGBM Modeling : {} seconds =====".format((time.time() - start_time)))   
    print(results)
    return results, y_probs, model

In [ ]:
sys.argv = ['scripts/train_model.py', 'P001', 1280, './cnn_model', 'cnn']
event_name = sys.argv[1]
feat_dim = sys.argv[2]
output_file = sys.argv[3]
model_type = sys.argv[4]

In [ ]:
train_mart = pd.read_csv('./datamart_cnn_train_avgpool.csv')
valid_mart = pd.read_csv('./datamart_cnn_valid_avgpool.csv')
test_mart  = pd.read_csv('./datamart_cnn_test_avgpool.csv')

In [ ]:
#Choose for each model based on MAP
#k = total_features.shape[1]
SVM_results_p001, _, cnn_model_p001 = modeling_ap_SVM(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002, _, cnn_model_p002 = modeling_ap_SVM(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003, _, cnn_model_p003 = modeling_ap_xgb(k=feat_dim, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
filename = os.path.join(output_file, event_name+'_'+model_type+'.model.sav')
filename

In [ ]:
pickle.dump(cnn_model_p001, open(filename, 'wb'))

In [ ]:
output_file = output_file = 'cnn_model/P001_cnn.model'
filename = output_file+'.sav'
pickle.dump(cnn_model_p001, open(filename, 'wb'))

output_file = output_file = 'cnn_model/P002_cnn.model'
filename = output_file+'.sav'
pickle.dump(cnn_model_p002, open(filename, 'wb'))

output_file = output_file = 'cnn_model/P003_cnn.model'
filename = output_file+'.sav'
pickle.dump(cnn_model_p003, open(filename, 'wb'))

In [ ]:
test_mart.shape[1]
test_mart.columns

In [ ]:
#python2 scripts/test_svm.py mfcc_pred/$event.model "kmeans/" $feat_dim_mfcc mfcc_pred/${event}_mfcc.lst || exit 1;
# sys.argv = ['scripts/test_model.py', 'P001', 'cnn_model', 1280, 'cnn_pred/P001_cnn.list', 'cnn']
sys.argv = ['scripts/test_model.py', 'P001', 'cnn_model', 1280, 'cnn']

### Test_model.py

In [ ]:
if __name__ == '__main__':
    if len(sys.argv) != 4:
        print("Usage: {0} model_file feat_dir feat_dim output_file".format(sys.argv[0]))
        print("model_file -- path of the trained svm file")
        print("feat_dir -- dir of feature files")
        print("output_file -- path to save the prediction score")
        print("model_type -- mfcc or asrs")
        exit(1)

    event_name = sys.argv[1]
    model_file = sys.argv[2]
    feat_dim = int(sys.argv[3])
    model_type = sys.argv[4]
    print(event_name)
    print(model_file)
    print(feat_dim)
    print(model_type)

In [ ]:
model.shape_fit_

In [ ]:
filename = os.path.join(model_file, event_name+'_'+model_type+'.model.sav')
model = pickle.load(open(filename, 'rb'))
test_mart  = pd.read_csv('./datamart_cnn_test_avgpool.csv')
X_test = test_mart.iloc[:, 1:feat_dim + 1]
scores = model.predict_proba(X_test)
scores_df = pd.DataFrame(scores)
scores_df.columns = ['N', 'Y']
scores_total = pd.concat([test_mart, scores_df], axis = 1)
# scores_total.head(3)

test_list = pd.read_csv('./list/test.video', header = None)
test_list.columns = ['video']

output_file = 'cnn_pred/'+event_name+'_'+model_type+'.list'
final_list = test_list.merge(scores_total, how = 'left', on = 'video')[['video', 'Y']]
# final_list.to_csv(output_file+'.video'+'.csv', index = False)
# final_list['Y'].to_csv(output_file+'.csv', index = False, header = True)
final_list.to_csv(output_file+'.video'+'.txt', index = False, header = False)
final_list['Y'].to_csv(output_file+'.txt', index = False, header = False)

In [ ]:
SVM_results_p001, _, cnn_model_p001 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
SVM_results_p002, _, cnn_model_p001 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
SVM_results_p003, _, cnn_model_p001 = modeling_ap_SVM(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
AdaB_results_p001, _, cnn_model_p001 = modeling_ap_AdaB(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
AdaB_results_p002, _, cnn_model_p001 = modeling_ap_AdaB(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
AdaB_results_p003, _, cnn_model_p001 = modeling_ap_AdaB(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
Boost_results_p001, _, cnn_model_p001 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Boost_results_p002, _, cnn_model_p001 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Boost_results_p003, _, cnn_model_p001 = modeling_ap_Boost(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
Xgb_results_p001, _, cnn_model_p001 = modeling_ap_xgb(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
Xgb_results_p002, _, cnn_model_p001 = modeling_ap_xgb(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
Xgb_results_p003, _, cnn_model_p001 = modeling_ap_xgb(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')

In [ ]:
LGBM_results_p001, _, cnn_model_p001 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p001_10')
LGBM_results_p002, _, cnn_model_p001 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p002_10')
LGBM_results_p003, _, cnn_model_p001 = modeling_ap_lgbm(k=k, train_data = train_mart, valid_data = valid_mart, target = 'target_p003_10')